In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.7MB/s 


In [2]:
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix-"

/content/drive/My Drive/Colab Notebooks/dw_matrix-


In [4]:
ls data

day3.ipynb  shoes_prices.csv


In [5]:
df = pd.read_csv('data/shoes_prices.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [8]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [9]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=1)
run_model(['brand_cat'], model)

(-57.285351528077264, 4.131927834435122)

In [10]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [11]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [12]:
test = {'key': 'value'}
test['key']

str(test)

"{'key': 'value'}"

In [13]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key']
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  
  return output_dict



df['features_parsed'] = df['features'].map(parse_features)

In [62]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

490

In [27]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [63]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_Number of Wheels', 'feat_Certifications and Listings',
       'feat_Item Style', 'feat_Waist Size', 'feat_Frame Color',
       'feat_brand_cat', 'feat_Color_cat', 'feat_Manufacturer Part Number_cat',
       'feat_Gender_cat', 'feat_Material_cat'],
      dtype='object', length=545)

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [65]:
{k:v for k,v in keys_stat.items() if v > 30}

{'Brand': 48.62691466083151,
 'Color': 47.784463894967175,
 'Gender': 50.10393873085339,
 'Manufacturer Part Number': 36.252735229759296,
 'Material': 34.18490153172867}

In [0]:
df['feat_brand_cat'] = df['feat_Brand'].factorize()[0]
df['feat_Color_cat'] = df['feat_Color'].factorize()[0]
df['feat_Manufacturer Part Number_cat'] = df['feat_Manufacturer Part Number'].factorize()[0]
df['feat_Gender_cat'] = df['feat_Gender'].factorize()[0]
df['feat_Material_cat'] = df['feat_Material'].factorize()[0]
df['feat_Sport_cat'] = df['feat_Sport'].factorize()[0]
df['feat_Style_cat'] = df['feat_Style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  get_name_feat(key)


In [71]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_Brand].shape

(8846, 545)

In [0]:
feats = ['']

In [75]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.31558895156295, 4.1246657105987845)

In [97]:
feats_cat = [ x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_Fabrication',
 'feat_Location - City/State',
 'feat_Multi Pack Indicator',
 'feat_Recommended Location',
 'feat_Location - Country',
 'feat_Certifications and Listings',
 'feat_brand_cat',
 'feat_Color_cat',
 'feat_Manufacturer Part Number_cat',
 'feat_Gender_cat',
 'feat_Material_cat',
 'feat_Sport_cat',
 'feat_Style_cat',
 'feat_Suitable For_cat',
 'feat_Fabrication_cat',
 'feat_Airport Friendly_cat',
 'feat_Number of Items_cat',
 'feat_Dial Color_cat',
 'feat_Characteristics_cat',
 'feat_Resolution_cat',
 'feat_Lens Material_cat',
 'feat_Assembled Product Weight_cat',
 'feat_Diamond Color_cat',
 'feat_Materials_cat',
 'feat_Rise_cat',
 'feat_Movement_cat',
 'feat_Fits Styles_cat',
 'feat_Purpose_cat',
 'feat_Safety Apparel Product Type_cat',
 'feat_Conflict_cat',
 'feat_Country of Origin - Components_cat',
 'feat_Dial Markers_cat',
 'feat_Cushioning_cat',
 'feat_Alarm_cat',
 'feat_Fabric Care_cat',
 'feat_Designed For_cat',
 'feat_Frame material:_

In [0]:
feats = ['brand_cat', 'feat_Gender_cat','feat_Material_cat','feat_Sport_cat','feat_Style_cat']
feats += ['feat_Compatible Devices_cat',
 'feat_Battery Voltage_cat',
 'feat_Cleaning, Care & Maintenance_cat',
 'feat_Sports Team_cat',
 'feat_Hammer loop_cat',
 'feat_Sleeve Length_cat',
 'feat_Upper Material_cat',
 'feat_Animal Type_cat',
 'feat_CM_cat',
 'feat_Mirrored_cat',
 'feat_Wheeled_cat',
 'feat_Tactical_cat',
 'feat_Enhanced Visibility (Reflective)_cat',
 'feat_Boot Height_cat',
 'feat_Nike Product #_cat',
 'feat_Weather-Resistant_cat',
 'feat_Color Mapping_cat',
 'feat_Walmart No._cat',
 'feat_Is Wheeled_cat',
 'feat_Safety Feature_cat',
 'feat_Sleeve length_cat',
 'feat_Closure_cat',
 'feat_Colour_cat',
 'feat_Sub Type_cat',
 'feat_Wheel Type_cat',
 'feat_Color/Finish Family_cat',
 'feat_Comfort Technology_cat',
 'feat_Target Audience_cat',
 'feat_Article_cat',
 'feat_Has Mercury_cat',
 'feat_Nike Id_cat',
 'feat_manufacturer_part_number_cat',
 'feat_Licensed_cat',
 'feat_Pronation_cat',
 'feat_Item weight unit of measure_cat',
 'feat_WIDTH_cat',
 'feat_atpv arc rating_cat',
 'feat_Age Gender Group_cat',
 'feat_Watch Shape_cat',
 'feat_Hardsided or Softsided_cat',
 'feat_Clasp Type_cat',
 'feat_Expandable_cat',
 'feat_Age Start_cat',
 'feat_Combined Shipping_cat',
 'feat_Multi Pack Indicator_cat',
 'feat_Fishing_cat',
 'feat_Manufacturer_cat',
 'feat_Heel Height_cat',
 'feat_Fits models_cat',
 'feat_Fit_cat',
 'feat_Stylenumber_cat',
 'feat_Hands_cat',
 'feat_Subdial_cat',
 'feat_Is Dark Sky-Compliant_cat',
 'feat_Neckline_cat',
 'feat_Fit:_cat',
 'feat_waterproof_cat',
 'feat_Chain Length_cat',
 'feat_Product Type_cat',
 'feat_Insulation_cat',
 'feat_Nickel free_cat',
 'feat_FIT_cat',
 'feat_Interior Pockets_cat',
 'feat_Stone Type_cat',
 'feat_Polarized?_cat',
 'feat_Frame Shape_cat',
 'feat_Digital Camera_cat',
 'feat_Front_cat',
 'feat_International Shipping_cat',
 'feat_Battery Type_cat',
 'feat_Water-Resistant_cat',
 'feat_Shipping Weight_cat',
 'feat_Use_cat',
 'feat_Motion Control_cat',
 'feat_Sock Size_cat',
 'feat_����������������_cat',
 'feat_Date first available at Amazon.ca_cat',
 'feat_Insulated_cat',
 'feat_Diameter_cat',
 'feat_Main Color_cat',
 'feat_Necklace Type_cat',
 'feat_Light Transmission (VLT)_cat',
 'feat_Watch Band Material_cat',
 'feat_Fits Most Screen Size_cat',
 'feat_Global Composite Sports Type_cat',
 'feat_Waist_cat',
 'feat_Casing_cat',
 'feat_Contains Batteries_cat',
 'feat_Instrument_cat',
 'feat_Occasion_cat',
 'feat_LENGTH_cat',
 'feat_Fits brands_cat',
 'feat_Item Package Quantity_cat',
 'feat_Bracelet Style_cat',
 'feat_Issued/ Not-Issued_cat',
 'feat_Sock Style_cat',
 'feat_Removable Hood_cat',
 'feat_Machine Washable_cat',
 'feat_Lens Color_cat',
 'feat_Sleeve Style_cat',
 'feat_Is Orthopedic_cat',
 'feat_Weather Resistant_cat',
 'feat_Profession_cat',
 'feat_MSRP_cat',
 'feat_Leg_cat',
 'feat_Boxed-product Dimensions_cat',
 'feat_Pocket_cat',
 'feat_Power Type_cat',
 'feat_Clothing Size_cat',
 'feat_Shape_cat',
 'feat_Mechanic_cat',
 'feat_Szie_cat',
 'feat_Manufacturer Number_cat',
 'feat_Used With Equipment Type_cat',
 'feat_Style #_cat',
 'feat_Temple Size:_cat',
 'feat_Display Technology_cat',
 'feat_Terrain_cat',
 'feat_FREE Item_cat',
 'feat_Model_cat',
 'feat_MPN_cat',
 'feat_sizebridge_cat',
 'feat_Band Length (in.)_cat',
 'feat_Polarised_cat',
 'feat_Assembled in Country of Origin_cat',
 'feat_To Fit_cat',
 'feat_Age End_cat',
 'feat_Style:_cat',
 'feat_Main Colour_cat',
 'feat_Case Type_cat',
 'feat_Type 2_cat',
 'feat_gender_cat',
 'feat_Attachment_cat',
 'feat_Chain Included_cat',
 'feat_Pattern_cat',
 'feat_Frame Type_cat',
 'feat_Inseam_cat',
 'feat_Dimensions_cat',
 'feat_Fits Model_cat',
 'feat_Has Expiration_cat',
 'feat_Is Polarized_cat',
 'feat_Number of Batteries Included_cat',
 'feat_Lens color family_cat',
 'feat_SHADE_cat',
 'feat_Accessory Type_cat',
 'feat_Case Back_cat',
 'feat_Rx-able_cat',
 'feat_Authenticity_cat',
 'feat_100% ADIDAS AUTHENTIC_cat',
 'feat_Features & Fastening_cat',
 'feat_Colour Code_cat',
 'feat_Retail Price_cat',
 'feat_Casual & Dress Shoe Style_cat',
 'feat_NAVY BLUE_cat',
 'feat_Quantity in Set_cat',
 'feat_Domestic Shipping_cat',
 'feat_Polarized_cat',
 'feat_Metal Color_cat',
 'feat_Lens_cat',
 'feat_Impact Resistant_cat',
 'feat_Lining_cat',
 'feat_Country of Manufacturer_cat',
 'feat_Is Energy Star-Certified_cat',
 'feat_GPS_cat',
 'feat_Lens Width_cat',
 'feat_Construction_cat',
 'feat_Is Lined_cat',
 'feat_Ground_cat',
 'feat_Width_cat',
 'feat_Feature_cat',
 'feat_Auto Shut-Off_cat',
 'feat_Vehicle Type_cat',
 'feat_Season_cat',
 'feat_Net Weight_cat',
 'feat_Protects Against_cat',
 'feat_Character_cat',
 'feat_Fabric Care Instructions_cat',
 'feat_Leather Grade_cat',
 'feat_Water Resistant_cat',
 'feat_spikes type_cat',
 'feat_Clothing Category_cat',
 'feat_Age Range_cat',
 'feat_Includes_cat',
 'feat_Band Material_cat',
 'feat_Looks best on these face shapes:_cat',
 'feat_Shoe Fastening Type_cat',
 'feat_Style Number_cat',
 'feat_Toe Type_cat',
 'feat_EURO SIZE_cat',
 'feat_Watch Power Source_cat',
 'feat_Is Weather-Resistant_cat',
 'feat_Lens Technology_cat',
 'feat_Gold Karat_cat',
 'feat_Band Manufacturer_cat',
 'feat_Country of Origin - Assembly_cat',
 'feat_Light Weight_cat',
 'feat_Number of Pockets_cat',
 'feat_Model Number_cat',
 'feat_MPN#_cat',
 'feat_Heat Zones_cat',
 'feat_Bag Size Range_cat',
 'feat_Model No._cat',
 'feat_Is Portable_cat',
 'feat_EAN_cat',
 'feat_Video Game Platform_cat',
 'feat_Release_cat',
 'feat_Product in Inches (L x W x H)_cat',
 'feat_Chain/Necklace Length (in.)_cat',
 'feat_Hat Style_cat',
 'feat_Part Number_cat',
 'feat_Case Thickness_cat',
 'feat_Height_cat',
 'feat_Is Recyclable_cat',
 'feat_Features_cat',
 'feat_Lining Material_cat',
 'feat_Genre_cat',
 'feat_Bridge_cat',
 'feat_Country////Region of Manufacture_cat',
 'feat_Hunting_cat',
 'feat_Product Name_cat',
 'feat_What Includes_cat',
 'feat_Black_cat',
 'feat_ADIDAS_cat',
 'feat_Case Diameter_cat',
 'feat_Crystal_cat',
 'feat_Case Tone_cat',
 'feat_Style Code_cat',
 'feat_Wash_cat',
 'feat_Handbag Style_cat',
 'feat_Fastener Type_cat',
 'feat_Lens Socket Width_cat',
 'feat_Item Spec Attr - Style (eBay)_cat',
 'feat_Band Type_cat',
 'feat_Fuel Type_cat',
 'feat_Shipping to USA_cat',
 'feat_Is Waterproof_cat',
 'feat_Variant Group ID_cat',
 'feat_Origin of Components_cat',
 'feat_Clothing Size Type_cat',
 'feat_Waterproof_cat',
 'feat_Batteries Included?_cat',
 'feat_Length_cat',
 'feat_Flame Resistant_cat',
 'feat_Country Of Origin_cat',
 'feat_Size_cat',
 'feat_Transactionid_cat',
 'feat_Watch Style_cat',
 'feat_Name_cat',
 'feat_Pattern or Design_cat',
 'feat_Labor_cat',
 'feat_Number of Pieces_cat',
 'feat_Batteries Required_cat',
 'feat_Applicable_cat',
 'feat_Auto_cat',
 'feat_Leg Opening:_cat',
 'feat_Front Style_cat',
 'feat_Parts_cat',
 'feat_Very popular bag now a days_cat',
 'feat_Capacity_cat',
 'feat_Shoe Category_cat',
 'feat_Lens material:_cat',
 'feat_Has Adaptive Lenses_cat',
 'feat_Catalog_cat',
 'feat_Recommended Location_cat',
 'feat_Overall Material_cat',
 'feat_Fabric Content_cat',
 'feat_Guaranteed Authentic_cat',
 'feat_Material Detail-1_cat',
 'feat_Type of Sole_cat',
 'feat_Number of Heat Settings_cat',
 'feat_High Visibility (ANSI Compliant)_cat',
 'feat_Jacket Length_cat',
 'feat_Place of Origin_cat',
 'feat_Resizable_cat',
 'feat_Case Material_cat',
 'feat_Product #_cat',
 'feat_Batteries Required?_cat',
 'feat_Hooded_cat',
 'feat_Band Width_cat',
 'feat_Location - Country_cat',
 'feat_Closure style_cat',
 'feat_Shoe Width_cat',
 'feat_Band Length_cat',
 'feat_Sole_cat',
 'feat_Date First Available_cat',
 'feat_Year_cat',
 'feat_Seller Inventory Number_cat',
 'feat_Eyewear Frame Style_cat',
 'feat_Fastener_cat',
 'feat_Shirt Size_cat',
 'feat_Year Made_cat',
 'feat_ISBN_cat',
 'feat_Case Finish_cat',
 'feat_Shoe Closure_cat',
 'feat_Gemstone Type_cat',
 'feat_General Warranty_cat',
 'feat_Audience_cat',
 'feat_Lens Type_cat',
 'feat_sizearm_cat',
 'feat_Label_cat',
 'feat_SKU_cat',
 'feat_Power Reserve_cat',
 'feat_Display_cat',
 'feat_Reinforced knee_cat',
 'feat_Adjustable_cat',
 'feat_Fits over eyeglasses_cat',
 'feat_UL Safety Listing_cat',
 'feat_Age Segment_cat',
 'feat_Bridge Size:_cat',
 'feat_Frame Style_cat',
 'feat_Shipping Weight (in pounds)_cat',
 'feat_Shoe Size_cat',
 'feat_amazonBestSellersRank_cat',
 'feat_Bezel_cat',
 'feat_Bridge/Temple Size:_cat',
 'feat_Watch Case Shape_cat',
 'feat_Clothing Type_cat',
 'feat_Shade_cat',
 'feat_Lens material_cat',
 'feat_Ring Style_cat',
 'feat_Full Product Manual_cat',
 'feat_Item weight_cat',
 'feat_Kids Backpacks_cat',
 'feat_Band Color_cat',
 'feat_Wind Resistant_cat',
 'feat_Number of Compartments_cat',
 'feat_Disclosures_cat',
 'feat_Material Composition_cat',
 'feat_Shoe Height_cat',
 'feat_Item color_cat',
 'feat_Military Style_cat',
 'feat_Face Shape_cat',
 'feat_Number of Wheels_cat',
 'feat_Certifications and Listings_cat',
 'feat_Item Style_cat',
 'feat_Waist Size_cat',
 'feat_Frame Color_cat']


model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [104]:
X = df[feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm , feature_names=feats)

(-57.614095702981295, 4.259369003792463)


Weight,Feature
0.2652 ± 0.0079,brand_cat
0.1070 ± 0.0072,feat_Material_cat
0.0266 ± 0.0015,feat_Gender_cat
0.0132 ± 0.0008,feat_Shoe Category_cat
0.0055 ± 0.0003,feat_Fabric Content_cat
0.0046 ± 0.0006,feat_Jacket Length_cat
0.0045 ± 0.0002,feat_Resizable_cat
0.0045 ± 0.0009,feat_Casual & Dress Shoe Style_cat
0.0037 ± 0.0013,feat_Item Package Quantity_cat
0.0026 ± 0.0007,feat_Watch Style_cat


In [0]:
df['weight'].unique()

In [93]:
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'Gender': 'boys', 'Shoe Category': 'boys shoes', 'Color': 'black/photo blue/volt/light magnet grey', 'Casual & Dress Shoe Style': 'athletic', 'Manufacturer Part Number': '307565053', 'Brand': 'nike', 'Shoe Width': 'medium'},
       {'Material': 'synthetic', 'Gender': 'men', 'Color': 'blue', 'Model': '704939 402', 'Manufacturer Part Number': '704939 402', 'Brand': 'nike', 'Age Group': 'adult'},
       {'Gender': 'men', 'Shoe Category': 'mens shoes', 'Color': 'white/metallic silver', 'Casual & Dress Shoe Style': 'running shoes', 'Manufacturer Part Number': '316376111', 'Brand': 'nike', 'Shoe Width': 'medium (d, m)'},
       {'Sport': 'running shoes', 'Occasion': 'running shoes', 'Material': 'mesh', 'Gender': 'men', 'Shoe Size': '11.5 us men', 'Shoe Category': "men's shoes", 'Fabric Content': 'mesh', 'Color': 'dusty cactus/volt/white', 'Model': '631303 330', 'Shoe Closure': 'lace-up', 'Casual & Dress Shoe Style': 'running shoes', 'Manufacturer Part Number': '631303 330', 'Variant 

In [94]:
df['fea_age group'].values()

KeyError: ignored